Required Packages:

In [57]:
# tensorflow==2.3.1
# nltk==3.5
# colorama==0.4.3
# numpy==1.18.5
# scikit_learn==0.23.2
# Flask==1.1.2

In [58]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

Now we load the json file and extract the required data.

In [59]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)
num_classes

17

The variable “training_sentences” holds all the training data (which are the sample messages in each intent category) and the “training_labels” variable holds all the target labels correspond to each training data.

Then we use “LabelEncoder()” function provided by scikit-learn to convert the target labels into a model understandable form

In [60]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

Next, we vectorize our text data corpus by using the “Tokenizer” class and it allows us to limit our vocabulary size up to some defined number. When we use this class for the text pre-processing task, by default all punctuations will be removed, turning the texts into space-separated sequences of words, and these sequences are then split into lists of tokens. They will then be indexed or vectorized. We can also add “oov_token” which is a value for “out of token” to deal with out of vocabulary words(tokens) at inference time.

In [61]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

The “pad_sequences” method is used to make all the training text sequences into the same size.

Model Training
Let’s define our Neural Network architecture for the proposed model and for that we use the “Sequential” model class of Keras.

In [62]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_4  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_12 (Dense)            (None, 16)                272       
                                                                 
 dense_13 (Dense)            (None, 16)                272       
                                                                 
 dense_14 (Dense)            (None, 17)                289       
                                                                 
Total params: 16833 (65.75 KB)
Trainable params: 16833 (65.75 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

Now we are ready to train our model. Simply we can call the “fit” method with training data and labels.

In [63]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500


3/3 [==============================] - 1s 6ms/step - loss: 2.8347 - accuracy: 0.0145
Epoch 2/500
3/3 [==============================] - 0s 7ms/step - loss: 2.8328 - accuracy: 0.0870
Epoch 3/500
3/3 [==============================] - 0s 6ms/step - loss: 2.8315 - accuracy: 0.1014
Epoch 4/500
3/3 [==============================] - 0s 7ms/step - loss: 2.8305 - accuracy: 0.1014
Epoch 5/500
3/3 [==============================] - 0s 7ms/step - loss: 2.8298 - accuracy: 0.1014
Epoch 6/500
3/3 [==============================] - 0s 8ms/step - loss: 2.8287 - accuracy: 0.1014
Epoch 7/500
3/3 [==============================] - 0s 7ms/step - loss: 2.8279 - accuracy: 0.1014
Epoch 8/500
3/3 [==============================] - 0s 6ms/step - loss: 2.8270 - accuracy: 0.1014
Epoch 9/500
3/3 [==============================] - 0s 6ms/step - loss: 2.8261 - accuracy: 0.1014
Epoch 10/500
3/3 [==============================] - 0s 5ms/step - loss: 2.8253 - accuracy: 0.1014
Epoch 11/500
3/3 [=======================

After training, it is better to save all the required files in order to use it at the inference time. So that we save the trained model, fitted tokenizer object and fitted label encoder object.

In [64]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets


Inference
Okay!!!! now it’s time to check how our model performs. 😊

We are going to implement a chat function to engage with a real user. When a new user message is received, the chatbot will calculate the similarity between the new text sequence and training data. Considering the confidence scores got for each category, it categorizes the user message to an intent with the highest confidence score.

In [65]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from fuzzywuzzy import fuzz

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        
        inp = input(Fore.LIGHTBLUE_EX + "You: " + Style.RESET_ALL)
        if inp.lower() == "quit":
            break
        
        print(Fore.LIGHTBLUE_EX + "You:" + Style.RESET_ALL, inp)  # Display user input

        best_similarity = -1
        best_response = None

        for intent in data['intents']:
            for idx, pattern in enumerate(intent['patterns']):
                if idx < len(intent['responses']):
                    similarity = fuzz.partial_ratio(inp.lower(), pattern.lower())
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_response = intent['responses'][idx]

        if best_similarity > 70 and best_response:  # Set a threshold for similarity
            print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, best_response)
        else:
            print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, "I'm sorry, I don't understand.")

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
You: hi
ChatBot: Hi :)
You: who are you
ChatBot: I'm Sonic, an Artificial Intelligent bot
You: what you doing
ChatBot: I do what you need in app?
You: Can you help?
ChatBot: Yes Sure, How can I support you
You: How do I connect an electrical socket
ChatBot: You need to correctly identify the wires and follow the appropriate wiring diagram. It's advisable to consult an electrician if you're not sure.
You: How can I fix a leak in the sink?
ChatBot: Close the main water valve before fixing a sink leak. It's important to act promptly.
You: How do I measure wood accurately for cutting?
ChatBot: Use a ruler and a measuring tape to accurately measure wood before cutting.
You: What are the reasons for water leakage in the bathroom?
ChatBot: Water leakage in the bathroom can be caused by damaged seals or loose joints. It needs to be fixed promptly.
You: What's the process for staining wood?
ChatBot: To stain wood, sand the surface, apply the sta